- author: Lee Meng
- date: 2018-08-17 18:00
- title: 如何用 Python 及 Airflow 輕鬆追漫畫連載
- slug: how-i-use-python-and-airflow-to-help-me-catch-up-with-latest-comic-with-ease
- tags: Python, Airflow, 資料工程, Selenium, Slack
- description: Airflow 是一個以 Python 開發的工作流管理系統，也是資料工程不可或缺的利器之一。近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構 ETL 以及處理批量資料的首選之一。這篇文章希望以一個簡易的漫畫連載通知 App 作為引子，讓讀者直觀地了解 Airflow 背後的運作原理、建立資料工程的知識基礎，並在閱讀本文後發揮自己的創意，實際應用 Airflow 來解決並自動化自己及企業的數據問題。
- summary: Airflow 是一個以 Python 開發的工作流管理系統，也是資料工程不可或缺的利器之一。近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構 ETL 以及處理批量資料的首選之一。這篇文章希望以一個簡易的漫畫連載通知 App 作為引子，讓讀者直觀地了解 Airflow 背後的運作原理、建立資料工程的知識基礎，並在閱讀本文後發揮自己的創意，實際應用 Airflow 來解決並自動化自己及企業的數據問題。
- image: raj-eiamworakul-603747-unsplash.jpg
- image_credit_url: https://unsplash.com/photos/o4c2zoVhjSw?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText
- status: draft

[Airflow](https://airflow.apache.org/) 是一個從 Airbnb 誕生並開源，以 [Python](https://www.python.org/) 寫成的[工作流管理系統（Workflow Management System）](https://zh.wikipedia.org/wiki/%E5%B7%A5%E4%BD%9C%E6%B5%81%E7%AE%A1%E7%90%86%E7%B3%BB%E7%BB%9F)，也是[各大企業](https://github.com/apache/incubator-airflow#who-uses-airflow)的資料工程環節中不可或缺的利器之一。

近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構穩定可靠的 ETL 以及定期處理批量資料的首選之一。（事實上在 [SmartNews](https://www.smartnews.com/en/)，除了 DS/DE，會使用 Airflow 的軟體工程師也不在少數）

儘管它的方便以及強大，在完全熟悉 Airflow 之前，因為有些專業術語以及資料工程概念的存在，不少初學者（包含當時的我）在剛開始的時候容易四處撞壁。另外如果一開始就以 ETL 當作 Airflow 的使用案例的話，未免過於枯燥乏味且缺少共鳴。

因此這篇文章希望以一個簡易的漫畫連載通知 App 作為引子，讓讀者能夠專注在了解 Airflow 的使用方式，以及學習如何建立一個工作流的基礎。如果你對資料工程有興趣，不太熟悉 Airflow 但有基本的 Python 程式基礎（或是純粹對追漫畫連載有興趣），我相信這篇文章應該會很適合你。

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/app.jpg" style="width:90%"/>
    </a>
    <p>Slack 截圖：追漫畫應該要是件輕鬆的事情。<br/>我們將利用 Airflow 及
        <a href="http://selenium-python.readthedocs.io/" target="_blank">Selenium </a>來實作一個像這樣會每天從 <a href="https://slack.com/" target="_blank">Slack</a> 推送最新漫畫連載的 App
    <br/>
    <br/>
</center>

想重新複習 ETL 概念的讀者可以參考先前的文章：[資料科學家為何需要了解資料工程](https://leemengtaiwan.github.io/why-you-need-to-learn-data-engineering-as-a-data-scientist.html#%E8%B3%87%E6%96%99%E7%AE%A1%E9%81%93)。

## 章節傳送門
為讓讀者完整了解開發這個 App 的背景脈絡、我是怎麼設計 App 邏輯以及選擇 Airflow 的原因，在實際開始寫 Python 之前有兩小節介紹。

如果你迫不及待，想馬上看怎麼用 Python 來實作，可以直接跳到 [Python 實作](#Python-實作)之後再回來查看前面章節。雖然我建議按照順序閱讀：）
- [所以為何要這 App ？](#所以為何要這-App-？)
- [為何要 Airflow ？](#為何要-Airflow-？)
- [Python 實作](#Python-實作)
- [結語](#結語)

## 所以為何要這 App ？

平常有在網路上追漫畫連載的讀者們應該都了解，市面上的漫畫網站通常都不是會員制的。更不用說「在新連載出的時候自動通知您！」這種推送功能（Push Notification）了。也因為這樣，導致我常常三不五時上去這些漫畫網站，看每個關注的漫畫到底出了最新一話了沒。可想而知，答案通常是否定的。（一週出一次每天檢查也沒用啊啊啊）

如果你只看海賊王一個漫畫（索隆好帥！），這或許沒什麼負擔。但就像上面 Slack 截圖顯示的，我不只關注海賊王，還看很多其他漫畫。讓事情更糟的是，到最後你會發現：
- 不記得自己到底在追哪些漫畫
- 每一部漫畫最後到底是看到第幾話
- 上一話是什麼時候出的
- 有幾話是新出而你還沒看的 

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/tim-gouw-68319-unsplash.jpg" style="width:90%"/>
    </a>
    <p>手動追最新連載經常讓我追到懷疑人生</p>
    <br/>
</center>

追漫畫連載應該要是個輕鬆且享受的事情。在一個人人會寫 code 的時代，何不自己做個 App 幫我們自動「檢查新連載」這件事情呢？

## 為何要 Airflow ？

概念上我們可以把此 App 需要做的工作按照「順序」由上往下列出來：
- 取得使用者的閱讀紀錄
- 去漫畫網站看有沒有新的章節
- 跟紀錄比較，有沒有新連載？
    - 沒有：
        - 什麼都不幹，結束
    - 有：
        - 寄 Slack 通知
        - 更新閱讀紀錄

我知道你在想什麼，要實作這個 App 的邏輯一點都不難。事實上我們已經完整定義這 App 的工作流了，只剩寫個 Python script 把每個工作的邏輯實作出來而已，為何需要 Airflow ？

在你往下滑前給個提示：這個 App 還有一個必要功能沒被提到。

<center>
        <img src="{filename}images/airflow/thought-2123970_1280.jpg" style="width:90%"/>
</center>

對！顯而易見的，這個 App 必須定期地、重複地去執行上述的工作流（Workflow）。

我們可不希望它只在明天早上去檢查新連載。我們希望它明天、下個月或是明年的今天都在運作。這也是為何我們需要一個像是 Airflow 的[工作流管理系統（Workflow Management System）](https://zh.wikipedia.org/wiki/%E5%B7%A5%E4%BD%9C%E6%B5%81%E7%AE%A1%E7%90%86%E7%B3%BB%E7%BB%9F)。Airflow 能幫我們：
- 排程工作流，定期執行
- 確保工作流內的工作按照順序被執行
- 各個工作失敗時自動重試（[墨菲定律](https://zh.wikipedia.org/wiki/%E6%91%A9%E8%8F%B2%E5%AE%9A%E7%90%86)）
- 簡單易懂的 Web UI 方便管理工作流

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/airflow.gif" style="width:90%"/>
    </a>
    <p>Airflow 的 Web UI 讓我們能更輕鬆地管理及排程工作</p> 
    <br/>
</center>

In [ ]:
Show me the code!!![](show-me-the-code.png)

## Python 實作

In [ ]:

—-
一步步開發 app, iteration 1,2, 3

從只有一個 dummy task(do everything here!) 的 一個 dag 開始
介紹 dag
測試單一 task 
測試整個 dag 

加所有dummy  task， 設定整個 dependencies 
介紹 xcom 
測試整個 dag

實際加功能進去
分每個 function 介紹

事後監控

看漫畫

## 為何要 Airflow？

In [ ]:
Why bother airflow?
你想要做的是資料處理，而不是建立 ＷＭＳ
* Why not python script? 墨菲定律， Retry automatically
* Schedule: Corn possible, but dependencies 難以管理
* why airflow not Azakaban? Apache incubator ，簡單 UI

在我們的漫畫 case 中 airflow 是一個 overkill, 但這篇文章希望透過go though 這個問題讓讀者熟悉 airflow 的運作，為解決企業等級的問題做好準備。

## 結語

In [ ]:


你甚至也可以選擇時用其它語言 (e.g., JAVA)


期望在讀者們能在閱讀文章之後實際應用 Airflow 來解決自己以及企業的問題。

「你也可以推薦漫畫給我」
不過老實說，我偶爾還是習慣動作會去檢查新的漫畫出來了沒。
- 結語： 你想自動化什麼？ implement it and tell me! 


之後將在奇幻旅程 2 提到我在 SmartNews 裡頭是怎麼使用 Airflow 的，歡迎訂閱